In [1]:
import tensorflow_datasets as tfds
import tensorflow as tf
import build_resnet

In [2]:
def format_image(image, label):
    image = tf.image.resize(image, (224, 224)) / 255.0
    return  image, label

In [3]:
(raw_train, raw_validation, raw_test), metadata = tfds.load(
    'cats_vs_dogs',
    split=['train[:80%]', 'train[80%:90%]', 'train[90%:]'],
    with_info=True,
    as_supervised=True,
)

In [4]:
num_examples = metadata.splits['train'].num_examples
num_classes = metadata.features['label'].num_classes
print(f"number of training examples:{num_examples}, number of classes: {num_classes}")

number of training examples:23262, number of classes: 2


In [5]:
BATCH_SIZE = 32
train_batches = raw_train.shuffle(num_examples // 4).map(format_image).batch(BATCH_SIZE).prefetch(1)
validation_batches = raw_validation.map(format_image).batch(BATCH_SIZE).prefetch(1)
test_batches = raw_test.map(format_image).batch(1)

for image_batch, label_batch in train_batches.take(1):
    pass

print(f"image shape: {image_batch.shape}")

image shape: (32, 224, 224, 3)


In [6]:
resnet = build_resnet.build_resnet()

In [7]:
resnet.compile(optimizer="adam",
              loss = 'sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [8]:
hist = resnet.fit(train_batches,
                 epochs=10,
                 validation_data=validation_batches)

Epoch 1/10
582/582 [==============================] - 74s 116ms/step - loss: 1.1454 - accuracy: 0.5628 - val_loss: 0.6606 - val_accuracy: 0.5877
Epoch 2/10
582/582 [==============================] - 67s 114ms/step - loss: 0.6204 - accuracy: 0.6581 - val_loss: 1.0047 - val_accuracy: 0.5718
Epoch 3/10
582/582 [==============================] - 65s 110ms/step - loss: 0.5455 - accuracy: 0.7258 - val_loss: 0.9648 - val_accuracy: 0.6040
Epoch 4/10
582/582 [==============================] - 64s 108ms/step - loss: 0.4830 - accuracy: 0.7731 - val_loss: 0.6210 - val_accuracy: 0.6569
Epoch 5/10
582/582 [==============================] - 64s 108ms/step - loss: 0.4187 - accuracy: 0.8098 - val_loss: 0.4979 - val_accuracy: 0.7605
Epoch 6/10
582/582 [==============================] - 63s 108ms/step - loss: 0.3689 - accuracy: 0.8379 - val_loss: 0.6154 - val_accuracy: 0.7334
Epoch 7/10
582/582 [==============================] - 63s 108ms/step - loss: 0.3168 - accuracy: 0.8648 - val_loss: 0.4765 - val_ac

In [9]:
resnet.evaluate(test_batches)

2326/2326 [==============================] - 35s 15ms/step - loss: 0.6576 - accuracy: 0.7661


[0.6575618982315063, 0.7661221027374268]